https://github.com/FinanceData/FinanceDataReader

In [5]:
import json
import datetime
import time
import pandas as pd
from pandas import json_normalize
import plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import FinanceDataReader as fdr

from requests_html import HTMLSession
session = HTMLSession()

In [2]:
def str_to_timestamp(date):
    obj_datetime = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return datetime_to_timestamp(obj_datetime)

def datetime_to_timestamp(dt):
    return int(time.mktime(dt.timetuple()))

def transform_ohlc(data):
    result = {}
    if 't' in data.keys():
        result['Date'] = data.pop('t')
        for i, v in enumerate(result['Date']):
            result['Date'][i] = datetime.datetime.fromtimestamp(v)
    if 'o' in data.keys():
        result['Open'] = data.pop('o')
    if 'h' in data.keys():
        result['High'] = data.pop('h')
    if 'l' in data.keys():
        result['Low'] = data.pop('l')
    if 'c' in data.keys():
        result['Close'] = data.pop('c')
    if 'v' in data.keys():
        result['Volume'] = data.pop('v')
        
    df = pd.DataFrame.from_dict(result)
    return df.set_index('Date')


host = 'https://tvc4.forexpros.com/bf9b0cf35ce6ca91b1bba8dd1d70c0c2/1601606035/18/18/88/'

def history(symbol, resolution, from_date, to_date=None):
    from_timestamp = str_to_timestamp(from_date + ' 00:00:00' if len(from_date) == 10 else from_date)
    if not to_date:
        to_timestamp = datetime_to_timestamp(datetime.datetime.now())
    else:     
        to_timestamp = str_to_timestamp(to_date + ' 23:59:59' if len(to_date) == 10 else to_date)

    uri = f'history?symbol={symbol}&resolution={resolution}&from={from_timestamp}&to={to_timestamp}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)
    return transform_ohlc(data)

def quotes(symbols):
    symbols_text = ','.join(symbols)
    uri = f'quotes?symbols={symbols_text}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)
    d = [row['v'] for row in data['d']]
    return pd.DataFrame(d)

def symbols(symbol):
    uri = f'symbols?symbol={symbol}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)    
    return json_normalize(data)

def search(query, limit=30):
    uri = f'search?limit={limit}&query={query}&type=&exchange='
    r = session.get(host + uri)
    data = json.loads(r.html.text)    
    return pd.DataFrame(data)
    

In [63]:
df = history(43433, 'D', '2020-01-01')

In [64]:
df['Close'].iplot(kind='line')

In [3]:
지수 = ['KS:KS11', 'KS:KS200', 'KQ:KQ11', 'T:JP225', 'AX:AXJ0', 'HK:HK50', 'SS:FTXIN9', 'NYSE:DJI', 'NYSE:US500',
     'NASDAQ:IXIC', 'P:US2000', 'DE:DE30', 'LON:UK100', 'DE:STOXX50', 'PA:F40', 'MI:IT40', 'TW:TWII', 'KA:KSE',
     'CM:CSE', 'BK:SETI']
주식 = ['NASDAQ:AAPL', 'NYSE:BABA', 'NYSE:TWTR', 'NYSE:AA', 'NYSE:BAC', 'NYSE:KO', 'NYSE:XOM', 'NASDAQ:FB',
     'NASDAQ:GOOGL', 'NYSE:HPQ', 'NYSE:JPM', 'NYSE:MCD', 'NASDAQ:MSFT', 'NASDAQ:SBUX']
선물 = ['COMEX:GC', 'NYMEX:CL', 'COMEX:SI', 'NYMEX:NG', 'CBOT:ZW', 'COMEX:HG', 'KS:KSU7', 'HK:HCEIc1', 'OSE:JP225',
     'Eurex:DE30', 'CME:US500', 'CME:USTEC']
외환 = ['USD/KRW', 'USD/JPY', 'JPY/KRW', 'EUR/USD', 'AUD/USD', 'itBit:BTC/USD', 'ICE:DX', 'EUR/KRW', 'GBP/USD',
     'EUR/JPY', 'USD/CHF', 'USD/CAD', 'GBP/JPY', 'NZD/USD', 'AUD/JPY', 'USD/CNY', 'MCX:USD/RUB']
종합 = ['COMEX:GC', 'COMEX:SI', 'NYMEX:CL', 'NYMEX:NG', 'NYMEX:USTEC', 'CME:USTEC', 'CME:US500', 
        'CBOT:US30', 'CBOT:ZS', 'CBOT:ZW', 'CBOE:VX', 'Eurex:DE30', 'OSE:JP225', 'KRX:KS', '서울:005930', 
        'USD/KRW', 'CHF/CAD', 'USD/CHF', 'USD/CAD']

In [4]:
quotes(선물)

,ch,chp,short_name,exchange,description,lp,ask,bid,spread,open_price,high_price,low_price,prev_close_price,volume
0,16.75,0.88,금,,금 선물,1911.85,"1,911.90","1,911.80",0,"1,898.10","1,913.40","1,898.10",1895.10,0
1,-0.07,-0.17,WTI유,,WTI유 선물,41.12,41.14,41.10,0,41.36,41.48,41.06,41.19,0
2,0.476,1.99,은,,은 선물,24.352,24.355,24.350,0,23.980,24.395,23.962,23.876,0
3,0.012,0.46,천연가스,,천연가스 선물,2.643,2.646,2.640,0,2.630,2.648,2.619,2.631,0
4,3.50,0.59,미국 소맥,,미국 소맥 선물,599.38,599.50,599.25,0,595.75,600.50,594.12,595.88,0
5,0.0290,0.95,구리,,구리 선물,3.0707,3.0710,3.0705,0,3.0427,3.0732,3.0393,3.0417,0
6,1.25,0.39,코스피200 선물 (F),,코스피200 선물 (F),318.95,319.20,318.70,0,319.70,319.80,316.60,318.95,0
7,-6.0,-0.06,홍콩 H,,홍콩 H 선물,9639.0,"9,650.0","9,628.0",0,"9,698.0","9,702.0","9,608.0",9645.0,0
8,-25.0,-0.11,닛케이,,닛케이 225 선물,23618.0,"23,630.0","23,606.0",0,"23,655.0","23,720.0","23,607.5",23643.0,"15,726"
9,-3.2,-0.02,독일 DAX,,독일 DAX 선물,13069.8,"13,073.8","13,065.8",0,"13,074.8","13,100.6","13,051.0",13073.0,"1,254"


In [19]:
symbols('KS:005930')

,name,exchange-traded,exchange-listed,timezone,minmov,minmov2,pricescale,pointvalue,has_intraday,has_no_volume,volume_precision,ticker,description,type,has_daily,has_weekly_and_monthly,supported_resolutions,intraday_multipliers,session,data_status
0,005930,서울,서울,Asia/Seoul,1,0,1,1,True,False,3,43433,"삼성전자, 한국",Stock,True,True,"[5, 15, 30, 60, 300, D, W, M, 45, 120, 240]","[1, 5, 15, 30, 60, 300]",2;0900-1530:23456,delayed_streaming


In [6]:
search('삼성')

,symbol,full_name,description,type,ticker,exchange
0,005930,서울:005930,삼성전자,주식,43433,서울
1,028260,서울:028260,삼성물산,주식,43477,서울
2,006400,서울:006400,삼성SDI,주식,43450,서울
3,032830,서울:032830,삼성생명,주식,43428,서울
4,000810,서울:000810,삼성화재해상보험,주식,43370,서울
5,207940,서울:207940,삼성바이오로직스,주식,993256,서울
6,018260,서울:018260,삼성에스디에스,주식,948394,서울
7,009150,서울:009150,삼성전기,주식,43350,서울
8,010140,서울:010140,삼성중공업,주식,43366,서울
9,028050,서울:028050,삼성엔지니어링,주식,43435,서울


In [4]:
tickers = {
    8874: {'symbol': 'USTEC', 'description': '나스닥 100 선물'}
}

{'symbol': 'USTEC', 'description': '나스닥 100 선물'}

In [5]:
targets = [
    ['삼성전자', 43433],
    ['SK하이닉스', 43430],
    ['네이버', 43493]
]

In [6]:
df_list = [history(ticker, 'D', '2019-01-01')['Close'] for name, ticker in targets]
df = pd.concat(df_list, axis=1)
df.columns = [name for name, ticker in targets] 
df.tail(10)

,삼성전자,SK하이닉스,네이버
Date,,,
2020-09-22 09:00:00,58200.0,81300.0,284500.0
2020-09-23 09:00:00,58600.0,83600.0,296500.0
2020-09-24 09:00:00,57800.0,84300.0,294000.0
2020-09-25 09:00:00,57900.0,82900.0,295000.0
2020-09-27 09:00:00,57900.0,82900.0,295000.0
2020-09-28 09:00:00,58200.0,82500.0,298500.0
2020-09-29 09:00:00,58200.0,84000.0,296500.0
2020-10-04 09:00:00,58200.0,84000.0,296500.0
2020-10-05 09:00:00,58700.0,83200.0,298500.0


In [7]:
df_plot = df / df.iloc[0] - 1.0
df_plot.iplot()

In [6]:
fdr.DataReader('005930')

,Open,High,Low,Close,Volume,Change
Date,,,,,,
1996-11-09,1107,1142,1103,1128,120160,NaN
1996-11-11,1128,1128,1086,1111,72590,-0.015071
1996-11-12,1128,1173,1128,1161,135350,0.045005
1996-11-13,1161,1162,1128,1132,97830,-0.024978
1996-11-14,1112,1131,1109,1111,69510,-0.018551
...,...,...,...,...,...,...
2020-09-29,58300,59000,58200,58200,15503563,0.000000
2020-10-05,57500,59200,57500,58700,20228289,0.008591
2020-10-06,59400,59900,58700,59000,14463826,0.005111


In [7]:
fdr.DataReader('AAPL', '2017')

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2017-01-03,29.04,28.95,29.08,28.69,115130000.0,0.0031
2017-01-04,29.00,28.96,29.13,28.94,84470000.0,-0.0014
2017-01-05,29.15,28.98,29.22,28.95,88770000.0,0.0052
2017-01-06,29.48,29.20,29.54,29.12,127010000.0,0.0113
2017-01-09,29.75,29.49,29.86,29.48,134250000.0,0.0092
...,...,...,...,...,...,...
2020-10-02,113.02,112.89,115.37,112.22,144710000.0,-0.0323
2020-10-05,116.50,113.91,116.65,113.55,106240000.0,0.0308
2020-10-06,113.16,115.80,116.09,112.30,161500000.0,-0.0287


In [8]:
fdr.DataReader('7203', '2020-01-01', exchange='TSE') # 소프트뱅크그룹(7203)

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2020-01-06,7565.0,7599.0,7626.0,7530.0,6670000.0,-0.0193
2020-01-07,7715.0,7654.0,7722.0,7619.0,4960000.0,0.0198
2020-01-08,7617.0,7565.0,7642.0,7541.0,5610000.0,-0.0127
2020-01-09,7670.0,7672.0,7690.0,7638.0,4250000.0,0.0070
2020-01-10,7661.0,7695.0,7698.0,7651.0,3480000.0,-0.0012
...,...,...,...,...,...,...
2020-10-05,6909.0,6978.0,6983.0,6886.0,4030000.0,0.0026
2020-10-06,6958.0,6909.0,6984.0,6887.0,4710000.0,0.0071
2020-10-07,7003.0,6928.0,7003.0,6904.0,4250000.0,0.0065


In [9]:
fdr.DataReader('ZG')

,Close,Open,High,Low,Volume,Change
Date,,,,,,
1979-12-27,515.50,517.00,517.00,513.00,0.0,0.0182
1979-12-28,517.80,516.00,517.80,510.40,0.0,0.0045
1979-12-31,533.60,527.88,534.50,527.88,0.0,0.0305
1980-01-02,575.50,562.50,577.00,558.00,7260.0,0.0785
1980-01-03,625.00,627.00,640.00,603.00,20750.0,0.0860
...,...,...,...,...,...,...
2020-10-05,1920.10,1910.90,1924.90,1891.20,177590.0,0.0066
2020-10-06,1908.80,1917.80,1927.00,1878.20,219440.0,-0.0059
2020-10-07,1890.80,1882.60,1902.40,1877.10,188590.0,-0.0094


In [10]:
fdr.StockListing('KRX')

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,이현우,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
2577,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2578,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2579,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
2580,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호, 염두성 (각자대표)",http://www.heerim.com,서울특별시


In [11]:
fdr.StockListing('S&P500')

,Symbol,Name,Sector,Industry
0,MMM,3M Company,Industrials,Industrial Conglomerates
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services
...,...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
502,ZBH,Zimmer Biomet Holdings,Health Care,Health Care Equipment
503,ZION,Zions Bancorp,Financials,Regional Banks


In [13]:
fdr.DataReader(['HSN1F', 'NASDAQCOM', 'M2'], data_source='fred') # HSN1F 주택판매지수, NASDAQCOM 나스닥종합지수 

,NASDAQCOM,HSN1F,M2
DATE,,,
1970-01-01,NaN,461.0,NaN
1970-02-01,NaN,373.0,NaN
1970-03-01,NaN,389.0,NaN
1970-04-01,NaN,445.0,NaN
1970-05-01,NaN,466.0,NaN
...,...,...,...
2020-10-02,11075.016000,1011.0,18652.6
2020-10-05,11332.484000,1011.0,18652.6
2020-10-06,11154.605000,1011.0,18652.6
